# Synthetic dataset

- use a simple base forecaster, the same formulation as the one in fable

In [1]:
from sktime.forecasting.sarimax import SARIMAX
from sktime.transformations.hierarchical.aggregate import Aggregator
from sktime.forecasting.reconcile import ReconcilerForecaster
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils._testing.hierarchical import _bottom_hier_datagen

import pandas as pd
import numpy as np

* load and save synthetic hierarchical data

In [2]:
y = _bottom_hier_datagen(
    no_bottom_nodes=10,
    no_levels=3,
    random_seed=111,
)

y.to_csv("./data/synthetic.csv")

y

passengers
l3_agg    l2_agg    l1_agg    timepoints             
l3_node01 l2_node02 l1_node07 1949-01      223.850000
                              1949-02      235.190000
                              1949-03      261.650000
                              1949-04      255.980000
                              1949-05      240.860000
...                                               ...
l3_node03 l2_node04 l1_node10 1960-08     7575.384848
                              1960-09     6256.786466
                              1960-10     5634.448079
                              1960-11     4707.619460
                              1960-12     5253.895625

[1440 rows x 1 columns]

* now aggregate for totals

In [3]:
agg = Aggregator(flatten_single_levels=False)
y = agg.fit_transform(y)
y

passengers
l3_agg    l2_agg    l1_agg    timepoints              
__total   __total   __total   1949-01      8712.748619
                              1949-02      9179.176121
                              1949-03     10268.938485
                              1949-04     10035.253113
                              1949-05      9412.530000
...                                                ...
l3_node03 l2_node04 l1_node10 1960-08      7575.384848
                              1960-09      6256.786466
                              1960-10      5634.448079
                              1960-11      4707.619460
                              1960-12      5253.895625

[2736 rows x 1 columns]

Train test split

* to be the same as [here](https://github.com/Nixtla/hierarchicalforecast/) I've left out the last 8 obs

In [4]:
y_train, y_test = temporal_train_test_split(y, test_size=8)
y_train

passengers
l3_agg    l2_agg    l1_agg    timepoints              
__total   __total   __total   1949-01      8712.748619
                              1949-02      9179.176121
                              1949-03     10268.938485
                              1949-04     10035.253113
                              1949-05      9412.530000
...                                                ...
l3_node03 l2_node04 l1_node10 1959-12      4902.042100
                              1960-01      5058.124287
                              1960-02      4720.557204
                              1960-03      5084.184449
                              1960-04      5634.448079

[2584 rows x 1 columns]

Fit the base forecasts

* use a simple AR1 model

In [5]:
h = 8
base_forecaster = SARIMAX(order=(1,0,0), trend="c", enforce_stationarity=False)
base_forecaster.fit(y_train)
prds = base_forecaster.predict(fh=np.arange(1, h + 1)).rename(columns={'passengers': 'base'})
prds

base
l3_agg    l2_agg    l1_agg    timepoints              
__total   __total   __total   1960-05     35921.349575
                              1960-06     35549.866696
                              1960-07     35192.536118
                              1960-08     34848.818684
                              1960-09     34518.195776
...                                                ...
l3_node03 l2_node04 l1_node10 1960-08      5392.437108
                              1960-09      5337.697505
                              1960-10      5285.085736
                              1960-11      5234.519088
                              1960-12      5185.918063

[152 rows x 1 columns]

Now fit the hierrachical reconciler forecasters

* note that each loop the base forecasters will be retrained
* we could do this a bit quicker for some reconciliation methods (Foecaster * Reconciler), but this is the easiest way

In [6]:
methods = sorted(ReconcilerForecaster.METHOD_LIST)

for method in methods:
    print(method)
    reconciler = ReconcilerForecaster(forecaster=base_forecaster, method=method)
    prds_recon = reconciler.fit_predict(y=y_train, fh=np.arange(1, h + 1)).rename(columns={'passengers': method})
    prds = pd.concat([prds, prds_recon], axis=1)

prds

bu
mint_cov
mint_shrink
ols
td_fcst
wls_str
wls_var


base            bu  \
l3_agg    l2_agg    l1_agg    timepoints                               
__total   __total   __total   1960-05     35921.349575  35921.231765   
                              1960-06     35549.866696  35549.641092   
                              1960-07     35192.536118  35192.212087   
                              1960-08     34848.818684  34848.404983   
                              1960-09     34518.195776  34517.700588   
...                                                ...           ...   
l3_node03 l2_node04 l1_node10 1960-08      5392.437108   5392.437108   
                              1960-09      5337.697505   5337.697505   
                              1960-10      5285.085736   5285.085736   
                              1960-11      5234.519088   5234.519088   
                              1960-12      5185.918063   5185.918063   

                                               mint_cov   mint_shrink  \
l3_agg    l2_agg    l1_agg    timepoints                                
__total   __total   __total   1960-05    -362218.067058  35921.345918   
                              1960-06    -358466.120035  35549.859692   
                              1960-07    -354857.158260  35192.526058   
                              1960-08    -351385.732597  34848.805838   
                              1960-09    -348046.601580  34518.180398   
...                                                 ...           ...   
l3_node03 l2_node04 l1_node10 1960-08          1.721545   5392.479406   
                              1960-09         -1.061952   5337.748124   
                              1960-10         -3.705786   5285.143891   
                              1960-11         -6.216589   5234.584046   
                              1960-12         -8.600691   5185.989142   

                                                   ols       td_fcst  \
l3_agg    l2_agg    l1_agg    timepoints                               
__total   __total   __total   1960-05     35921.340851  35921.349575   
                              1960-06     35549.849985  35549.866696   
                              1960-07     35192.512108  35192.536118   
                              1960-08     34848.788019  34848.818684   
                              1960-09     34518.159058  34518.195776   
...                                                ...           ...   
l3_node03 l2_node04 l1_node10 1960-08      5392.511763   5392.536517   
                              1960-09      5337.786855   5337.816418   
                              1960-10      5285.188398   5285.222296   
                              1960-11      5234.633772   5234.671566   
                              1960-12      5186.043567   5186.084848   

                                               wls_str       wls_var  
l3_agg    l2_agg    l1_agg    timepoints                              
__total   __total   __total   1960-05     35921.348268  35921.349546  
                              1960-06     35549.864193  35549.866640  
                              1960-07     35192.532520  35192.536038  
                              1960-08     34848.814088  34848.818581  
                              1960-09     34518.190272  34518.195653  
...                                                ...           ...  
l3_node03 l2_node04 l1_node10 1960-08      5392.516453   5392.490331  
                              1960-09      5337.792465   5337.761201  
                              1960-10      5285.194842   5285.158917  
                              1960-11      5234.640968   5234.600835  
                              1960-12      5186.051438   5186.007517  

[152 rows x 8 columns]

Calculate the RMSE

* this is much the same as the fable package
* note that the covariance matrix for the `mint_cov` is not pos def here probably
    - should probably put a check in here on sktime

In [7]:
eval = pd.concat([prds, y_test], axis=1)
eval = eval.melt(id_vars='passengers', ignore_index=False)

eval.groupby('variable').apply(
    lambda x: np.round((((x['passengers'] - x['value'])**2).mean())**(1/2), 3)
)

variable
base             2601.781
bu               2601.857
mint_cov       496458.562
mint_shrink      2601.781
ols              2601.781
td_fcst          2601.771
wls_str          2601.776
wls_var          2601.776
dtype: float64